# Second dose first day with respect to first dose efficacy

In [ ]:
import os
import pandas as pd
import numpy as np
import datetime
import itertools
import matplotlib.pylab as plt
from matplotlib import rc
rc("text", usetex=True)
rc("font", family="serif")

import run_robot
import prepare_data
from importlib import reload
reload(prepare_data)
reload(run_robot)
from julia import Main as Julia 

In [ ]:
# Define the basic data for the case studies

# Mean incubation time
tinc = 5.2

# Mean inffectious time
tinf = 2.9

# Basic reproduction number
basic_rep = 2.5

# Lockdown level
lock_level = 1.0

# Simulation horizon
# A little more than a year when thevaccine should be here
ndays = 7*52

# Mean time in ICU
time_icu = 7

# Transmission/infection effect of the vaccine 
r_atten = [1.0, 0.24, 0.176]

# Symptoms effect of the vaccine
icu_atten = [1.0, 1.0, 1.0]

# R0 factor for each population
r0pop = [1.0, 1.3, 1.0, 1.0]

# Max dose profile
max_doses = 0.015*np.ones(ndays)
max_doses[0:30] = 0.001
max_doses[30:150] = 0.005

# Define basic paramters
basic_prm = prepare_data.save_basic_parameters(tinc=tinc, tinf=tinf, rep=basic_rep, min_level=lock_level,
    time_icu=time_icu, alternate=0, window=14, ndays=ndays)

pre_cities_data = {
    "S1": [0.685], 
    "E1": [0.01], 
    "I1": [0.006], 
    "R1": [0.299], 
    "population": [44639899], 
    "icu_capacity": [0.000175], 
    "start_date": [pd.to_datetime("2020-2-1")]
}
cities_data = pd.DataFrame(pre_cities_data, index=["SP"])
cities_data

In [ ]:
# Create a target matrix (max infected level)
ncities, ndays = len(cities_data.index), int(basic_prm["ndays"])
target = 0.8*np.ones((ncities, ndays))
target = prepare_data.save_target(cities_data, target)

# Use a forcedif that releases the cities in the end
force_dif = np.zeros((ncities, ndays))

In [ ]:
# Add information on the time series that estimate the need of ICUs
# We are using the time series adjusted considering that the mean ICU stay is 7 days (which lead to larger ICU capacity).

# Define the time series to estimate ICU demand
ts_sp = np.array([0.009, 0.015, 0.492, 0.0, 0.0, 0.0, 0.083, 0.011, 0.011])

ts_drs = ts_sp*np.ones((len(cities_data), len(ts_sp)))
ts_drs = pd.DataFrame(data=ts_drs, index=cities_data.index, columns=[
    "rho_min", "rho_max", "intercept", "trend", "phi_1", "phi_2", "sigma_omega", "state0", "state_less_1"
])
ts_drs["confidence"] = 0.9
ts_drs["time_icu"] = time_icu
cities_data = pd.concat([cities_data, ts_drs], axis=1)
cities_data

In [ ]:
pd.set_option("display.width", 120)

# Simple function to run a test and save results
def run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, r0pop, r_atten, icu_atten, max_doses, verbosity=1):
    run_robot.prepare_optimization(basic_prm, cities_data, M, target, hammer_data, force_dif,
        r0pop, r_atten, icu_atten, max_doses, verbosity=verbosity)
    run_robot.optimize_and_show_results(basic_prm, figure_file, result_file, cities_data, target, verbosity=verbosity)

In [ ]:
# Define mobility matrix.
M = prepare_data.convert_mobility_matrix_and_save(cities_data, max_neighbors=5, drs="data/report_drs_mobility.csv")
hammer_data = prepare_data.save_hammer_data(cities_data, 0, basic_prm["min_level"])
run_robot.find_feasible_hammer(basic_prm, cities_data, M, target, hammer_data, out_file=None, 
    incr_all=True, verbosity=1)
M

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def agregate(solution, labels):
    values = solution.loc[labels[0]].sum()
    for i in range(1, len(labels)):
        values += solution.loc[labels[i]].sum()
    return values

def retrieve_vaccination_profile(sol, pop=0, dose=0):
    sol = sol.loc[:, pop, :]
    seir = sol.loc["s", :] + sol.loc["e", :] + sol.loc["i", :] + sol.loc["r", :]
    applied = sol.loc["v", dose]*seir.loc[dose]
    return applied[14:]

## With vaccination blocking infection

In [ ]:
print("******************** Running vaccine simulation")
icu_atten = [1.0, 1.0, 1.0]
bins = [[0, 19], [20, 49], [50, 64], [65, 200]]
figure_all_file = "results/efficacy_sensitivity_infection_R0_20_49_all.svg"
figure_all = plt.figure()

basic_prm["alternate"] = 0.0
for difr0 in np.linspace(1, 1.6, 7):
    r0pop[1] = difr0
    dose_delay = []
    for atten in np.linspace(1.0, 0.20, 17):
        r_atten = [1.0, atten, 0.176]
        figure_file = f"results/efficacy_sensitivity_infection_R0_20_49_{difr0:.2f}"
        result_file =  figure_file + f"_atten_{atten:.2f}.csv"
        run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, r0pop, r_atten, icu_atten, max_doses)
        if not Julia.opt_OK:
            print("Problem in optimization for", r0pop, r_atten)
            print("Living result out of plot")
            continue

        solution = pd.read_csv(result_file, index_col=[0, 1, 2])

        total_dose1 = retrieve_vaccination_profile(solution, 0, 0)
        for i in range(1, len(bins)):
            total_dose1 += retrieve_vaccination_profile(solution, i, 0)
        total_dose2 = retrieve_vaccination_profile(solution, 0, 1)
        for i in range(1, len(bins)):
            total_dose2 += retrieve_vaccination_profile(solution, i, 1)

        start_dose1 = (total_dose1.cumsum() >= 0.2*max_doses[0]).argmax()
        start_dose2 = (total_dose2.cumsum() >= 0.2*max_doses[0]).argmax()
        dose_delay.append((100*(1.0 - atten), start_dose2 - start_dose1))
        print(r0pop, ":", dose_delay)
    dose_delay = np.array(dose_delay).T
    plt.figure()
    plt.plot(dose_delay[0], dose_delay[1], marker="o", lw=3)
    plt.xlabel("First dose efficacy")
    plt.ylabel("Days to second dose")
    plt.title("Time to second dose (from 28-84) - block infection")
    plt.axis("tight")
    plt.savefig(figure_file + ".svg")
    plt.figure(figure_all.number)
    plt.plot(dose_delay[0], dose_delay[1], marker="o", lw=3, label=f"20-50 R0 factor {difr0:.1f}")
plt.xlabel("First dose efficacy")
plt.ylabel("Days to second dose")
plt.title("Time to second dose (from 28-84) - block infection")
plt.legend()
plt.axis("tight")
plt.savefig(figure_all_file)

## With vaccination decreasing symptoms

In [ ]:
print("******************** Running vaccine simulation")
r_atten = [1.0, 1.0, 1.0]
bins = [[0, 19], [20, 49], [50, 64], [65, 200]]
figure_all_file = "results/efficacy_sensitivity_symptoms_R0_20_49_all.svg"
figure_all = plt.figure()

basic_prm["alternate"] = 0.0
for difr0 in np.linspace(1, 1.6, 7):
    r0pop[1] = difr0
    dose_delay = []
    for atten in np.linspace(1.0, 0.20, 17):
        icu_atten = [1.0, atten, 0.176]
        figure_file = f"results/efficacy_sensitivity_symptoms_R0_20_49_{difr0:.2f}"
        result_file =  figure_file + f"_atten_{atten:.2f}.csv"
        run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, r0pop, r_atten, icu_atten, max_doses)
        if not Julia.opt_OK:
            print("Problem in optimization for", r0pop, r_atten)
            print("Living result out of plot")
            continue

        solution = pd.read_csv(result_file, index_col=[0, 1, 2])

        total_dose1 = retrieve_vaccination_profile(solution, 0, 0)
        for i in range(1, len(bins)):
            total_dose1 += retrieve_vaccination_profile(solution, i, 0)
        total_dose2 = retrieve_vaccination_profile(solution, 0, 1)
        for i in range(1, len(bins)):
            total_dose2 += retrieve_vaccination_profile(solution, i, 1)

        start_dose1 = (total_dose1.cumsum() >= 0.2*max_doses[0]).argmax()
        start_dose2 = (total_dose2.cumsum() >= 0.2*max_doses[0]).argmax()
        dose_delay.append((100*(1.0 - atten), start_dose2 - start_dose1))
        print(r0pop, ":", dose_delay)
    plt.figure()
    dose_delay = np.array(dose_delay).T
    plt.plot(dose_delay[0], dose_delay[1], marker="o", lw=3)
    plt.xlabel("First dose efficacy")
    plt.ylabel("Days to second dose")
    plt.title("Time to second dose (from 28-84) - decrease symptoms")
    plt.axis("tight")
    plt.savefig(figure_file + ".svg")
    plt.figure(figure_all.number)
    plt.plot(dose_delay[0], dose_delay[1], marker="o", lw=3, label=f"20-50 R0 factor {difr0:.1f}")
plt.xlabel("First dose efficacy")
plt.ylabel("Days to second dose")
plt.title("Time to second dose (from 28-84) - block symptoms")
plt.legend()
plt.axis("tight")
plt.savefig(figure_all_file)


## Blocking infection and varying the basic reproduction number from 1.8 - 3.0

In [ ]:
print("******************** Running vaccine simulation")
icu_atten = [1.0, 1.0, 1.0]
bins = [[0, 19], [20, 49], [50, 64], [65, 200]]
r0pop = [1.0, 1.3, 1.0, 1.0]
figure_all_file = "results/efficacy_sensitivity_infection_basic_R0_all.svg"
figure_all = plt.figure()


basic_prm["alternate"] = 0.0
for basic_rep in [2.5]: #np.linspace(1.8, 3.0, 7):
    basic_prm["rep"] = basic_rep
    dose_delay = []
    for atten in np.linspace(1.0, 0.20, 17):
        r_atten = [1.0, atten, 0.176]
        figure_file = f"results/efficacy_sensitivity_infection_basic_rep_{basic_rep:.2f}"
        result_file =  figure_file + f"_atten_{atten:.2f}.csv"
        run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, r0pop, r_atten, icu_atten, max_doses)
        if not Julia.opt_OK:
            print("Problem in optimization for", r0pop, r_atten)
            print("Living result out of plot")
            continue

        solution = pd.read_csv(result_file, index_col=[0, 1, 2])

        total_dose1 = retrieve_vaccination_profile(solution, 0, 0)
        for i in range(1, len(bins)):
            total_dose1 += retrieve_vaccination_profile(solution, i, 0)
        total_dose2 = retrieve_vaccination_profile(solution, 0, 1)
        for i in range(1, len(bins)):
            total_dose2 += retrieve_vaccination_profile(solution, i, 1)

        start_dose1 = (total_dose1.cumsum() >= 0.2*max_doses[0]).argmax()
        start_dose2 = (total_dose2.cumsum() >= 0.2*max_doses[0]).argmax()
        dose_delay.append((100*(1.0 - atten), start_dose2 - start_dose1))
        print(basic_rep, ":", dose_delay)
    dose_delay = np.array(dose_delay).T
    plt.figure()
    plt.plot(dose_delay[0], dose_delay[1], marker="o", lw=3)
    plt.xlabel("First dose efficacy")
    plt.ylabel("Days to second dose")
    plt.title("Time to second dose (from 28-84) - block infection")
    plt.axis("tight")
    plt.savefig(figure_file + ".svg")
    plt.figure(figure_all.number)
    plt.plot(dose_delay[0], dose_delay[1], marker="o", lw=3, label=f"R0 factor {basic_rep:.2f}")
plt.xlabel("First dose efficacy")
plt.ylabel("Days to second dose")
plt.title("Time to second dose (from 28-84) - block infection")
plt.legend()
plt.axis("tight")
plt.savefig(figure_all_file)


## Blocking symptoms and varying the basic reproduction number from 1.8 - 3.0

In [ ]:
print("******************** Running vaccine simulation")
r_atten = [1.0, 1.0, 1.0]
bins = [[0, 19], [20, 49], [50, 64], [65, 200]]
r0pop = [1.0, 1.3, 1.0, 1.0]
figure_all_file = "results/efficacy_sensitivity_symptoms_basic_R0_all.svg"
figure_all = plt.figure()


basic_prm["alternate"] = 0.0
for basic_rep in [2.5]: # np.linspace(1.8, 3.0, 7):
    basic_prm["rep"] = basic_rep
    dose_delay = []
    for atten in np.linspace(1.0, 0.20, 17):
        icu_atten = [1.0, atten, 0.176]
        figure_file = f"results/efficacy_sensitivity_symptoms_basic_rep_{basic_rep:.2f}"
        result_file =  figure_file + f"_atten_{atten:.2f}.csv"
        run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, r0pop, r_atten, icu_atten, max_doses)
        if not Julia.opt_OK:
            print("Problem in optimization for", r0pop, r_atten)
            print("Living result out of plot")
            continue

        solution = pd.read_csv(result_file, index_col=[0, 1, 2])

        total_dose1 = retrieve_vaccination_profile(solution, 0, 0)
        for i in range(1, len(bins)):
            total_dose1 += retrieve_vaccination_profile(solution, i, 0)
        total_dose2 = retrieve_vaccination_profile(solution, 0, 1)
        for i in range(1, len(bins)):
            total_dose2 += retrieve_vaccination_profile(solution, i, 1)

        start_dose1 = (total_dose1.cumsum() >= 0.2*max_doses[0]).argmax()
        start_dose2 = (total_dose2.cumsum() >= 0.2*max_doses[0]).argmax()
        dose_delay.append((100*(1.0 - atten), start_dose2 - start_dose1))
        print(basic_rep, ":", dose_delay)
    dose_delay = np.array(dose_delay).T
    plt.figure()
    plt.plot(dose_delay[0], dose_delay[1], marker="o", lw=3)
    plt.xlabel("First dose efficacy")
    plt.ylabel("Days to second dose")
    plt.title("Time to second dose (from 28-84) - decrease symptoms")
    plt.axis("tight")
    plt.savefig(figure_file + ".svg")
    plt.figure(figure_all.number)
    plt.plot(dose_delay[0], dose_delay[1], marker="o", lw=3, label=f"R0 factor {basic_rep:.2f}")
plt.xlabel("First dose efficacy")
plt.ylabel("Days to second dose")
plt.title("Time to second dose (from 28-84) - block symptoms")
plt.legend()
plt.axis("tight")
plt.savefig(figure_all_file)